In [2]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sams
import holodeck.gravwaves
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode

log = holo.log
log.setLevel(logging.INFO)

# Plotting Methods

In [3]:
def plot_bin_pop(pop):
    mt, mr = utils.mtmr_from_m1m2(pop.mass)
    redz = cosmo.a_to_z(pop.scafa)
    data = [mt/MSOL, mr, pop.sepa/PC, 1+redz]
    data = [np.log10(dd) for dd in data]
    reflect = [None, [None, 0], None, [0, None]]
    labels = [r'M/M_\odot', 'q', r'a/\mathrm{{pc}}', '1+z']
    labels = [r'${{\log_{{10}}}} \left({}\right)$'.format(ll) for ll in labels]

    if pop.eccen is not None:
        data.append(pop.eccen)
        reflect.append([0.0, 1.0])
        labels.append('e')

    kde = kale.KDE(data, reflect=reflect)
    corner = kale.Corner(kde, labels=labels, figsize=[8, 8])
    corner.plot_data(kde)
    return corner


def plot_mbh_scaling_relations(pop, fname=None, color='r'):
    units = r"$[\log_{10}(M/M_\odot)]$"
    fig, ax = plt.subplots(figsize=[8, 5])
    ax.set(xlabel=f'Stellar Mass {units}', ylabel=f'BH Mass {units}')

    #   ====    Plot McConnell+Ma-2013 Data    ====
    handles = []
    names = []
    if fname is not None:
        hh = _draw_MM2013_data(ax, fname)
        handles.append(hh)
        names.append('McConnell+Ma')

    #   ====    Plot MBH Merger Data    ====
    hh, nn = _draw_pop_masses(ax, pop, color)
    handles = handles + hh
    names = names + nn
    ax.legend(handles, names)

    return fig


def _draw_MM2013_data(ax):
    data = holo.observations.load_mcconnell_ma_2013()
    data = {kk: data[kk] if kk == 'name' else np.log10(data[kk]) for kk in data.keys()}
    key = 'mbulge'
    mass = data['mass']
    yy = mass[:, 1]
    yerr = np.array([yy - mass[:, 0], mass[:, 2] - yy])
    vals = data[key]
    if np.ndim(vals) == 1:
        xx = vals
        xerr = None
    elif vals.shape[1] == 2:
        xx = vals[:, 0]
        xerr = vals[:, 1]
    elif vals.shape[1] == 3:
        xx = vals[:, 1]
        xerr = np.array([xx-vals[:, 0], vals[:, 2]-xx])
    else:
        raise ValueError()

    idx = (xx > 0.0) & (yy > 0.0)
    if xerr is not None:
        xerr = xerr[:, idx]
    ax.errorbar(xx[idx], yy[idx], xerr=xerr, yerr=yerr[:, idx], fmt='none', zorder=10)
    handle = ax.scatter(xx[idx], yy[idx], zorder=10)
    ax.set(ylabel='MBH Mass', xlabel=key)

    return handle


def _draw_pop_masses(ax, pop, color='r', nplot=3e3):
    xx = pop.mbulge.flatten() / MSOL
    yy_list = [pop.mass]
    names = ['new']
    if hasattr(pop, '_mass'):
        yy_list.append(pop._mass)
        names.append('old')

    colors = [color, '0.5']
    handles = []
    if xx.size > nplot:
        cut = np.random.choice(xx.size, int(nplot), replace=False)
        print("Plotting {:.1e}/{:.1e} data-points".format(nplot, xx.size))
    else:
        cut = slice(None)

    for ii, yy in enumerate(yy_list):
        yy = yy.flatten() / MSOL
        data = np.log10([xx[cut], yy[cut]])
        kale.plot.dist2d(
            data, ax=ax, color=colors[ii], hist=False, contour=True,
            median=True, mask_dense=True,
        )
        hh, = plt.plot([], [], color=colors[ii])
        handles.append(hh)

    return handles, names


def plot_gwb(gwb, color=None, uniform=False, nreals=5):
    """Plot a GW background from the given `Grav_Waves` instance.

    Plots samples, confidence intervals, power-law, and adds twin-Hz axis (x2).

    Parameters
    ----------
    gwb : `gravwaves.Grav_Waves` (subclass) instance

    Returns
    -------
    fig : `mpl.figure.Figure`
        New matplotlib figure instance.

    """

    fig, ax = plot.figax(
        scale='log',
        xlabel=r'frequency $[\mathrm{yr}^{-1}]$',
        ylabel=r'characteristic strain $[\mathrm{h}_c]$'
    )

    if uniform:
        color = ax._get_lines.get_next_color()

    _draw_gwb_sample(ax, gwb, color=color, num=nreals)
    _draw_gwb_conf(ax, gwb, color=color)
    plot._draw_plaw(ax, gwb.freqs*YR, f0=1, color='0.5', lw=2.0, ls='--')

    plot._twin_hz(ax, nano=True, fs=12)
    return fig


def _draw_gwb_sample(ax, gwb, num=10, back=True, fore=True, color=None):
    back_flag = back
    fore_flag = fore
    back = gwb.back
    fore = gwb.fore

    freqs = gwb.freqs * YR
    pl = dict(alpha=0.5, color=color, lw=0.8)
    plsel = dict(alpha=0.85, color=color, lw=1.6)
    sc = dict(alpha=0.25, s=20, fc=color, lw=0.0, ec='none')
    scsel = dict(alpha=0.50, s=40, ec='k', fc=color, lw=1.0)

    cut = np.random.choice(back.shape[1], num, replace=False)
    sel = cut[0]
    cut = cut[1:]

    color_gen = None
    color_sel = None
    if back_flag:
        hands_gen = ax.plot(freqs, back[:, cut], **pl)
        hands_sel, = ax.plot(freqs, back[:, sel], **plsel)
        color_gen = [hh.get_color() for hh in hands_gen]
        color_sel = hands_sel.get_color()

    if color is None:
        sc['fc'] = color_gen
        scsel['fc'] = color_sel

    if fore_flag:
        yy = fore[:, cut]
        xx = freqs[:, np.newaxis] * np.ones_like(yy)
        dx = np.diff(freqs)
        dx = np.concatenate([[dx[0]], dx])[:, np.newaxis]

        dx *= 0.2
        xx += np.random.normal(0, dx, np.shape(xx))
        # xx += np.random.uniform(-dx, dx, np.shape(xx))
        xx = np.clip(xx, freqs[0]*0.75, None)
        ax.scatter(xx, yy, **sc)

        yy = fore[:, sel]
        xx = freqs
        ax.scatter(xx, yy, **scsel)

    return


def _draw_gwb_conf(ax, gwb, **kwargs):
    conf = [0.25, 0.50, 0.75]
    freqs = gwb.freqs * YR
    back = gwb.back
    kwargs.setdefault('alpha', 0.5)
    kwargs.setdefault('lw', 0.5)
    conf = np.percentile(back, 100*np.array(conf), axis=-1)
    ax.fill_between(freqs, conf[0], conf[-1], **kwargs)
    kwargs['alpha'] = 1.0 - 0.5*(1.0 - kwargs['alpha'])
    ax.plot(freqs, conf[1], **kwargs)
    return


def plot_evo(evo, freqs=None, sepa=None, ax=None):
    if (freqs is None) and (sepa is None):
        err = "Either `freqs` or `sepa` must be provided!"
        log.exception(err)
        raise ValueError(err)

    if freqs is not None:
        data = evo.at('fobs', freqs)
        xx = freqs * YR
        xlabel = 'GW Frequency [1/yr]'
    else:
        data = evo.at('sepa', sepa)
        xx = sepa / PC
        xlabel = 'Binary Separation [pc]'

    if ax is None:
        fig, ax = plot.figax(xlabel=xlabel)
    else:
        fig = ax.get_figure()

    def _draw_vals_conf(ax, xx, vals, color=None, label=None):
        if color is None:
            color = ax._get_lines.get_next_color()
        if label is not None:
            ax.set_ylabel(label, color=color)
            ax.tick_params(axis='y', which='both', colors=color)
        # vals = np.percentile(vals, [25, 50, 75], axis=0) / units
        vals = utils.quantiles(vals, [0.25, 0.50, 0.75], axis=0).T
        h1 = ax.fill_between(xx, vals[0], vals[-1], alpha=0.25, color=color)
        h2, = ax.plot(xx, vals[1], alpha=0.75, lw=2.0, color=color)
        return (h1, h2)

    # handles = []
    # labels = []

    name = 'Hardening Time [yr]'
    vals = np.fabs(data['sepa'] / data['dadt']) / YR
    _draw_vals_conf(ax, xx, vals, label=name)
    # handles.append(hh)
    # labels.append(name)

    # name = 'eccen'
    # tw = ax.twinx()
    # hh, nn = _draw_vals_conf(tw, freqs*YR, name, 'green')
    # if hh is not None:
    #     handles.append(hh)
    #     labels.append(nn)

    # ax.legend(handles, labels)
    return fig

# Quick-Start

Create an Illustris-Based Population, and a simple binary-evolution model

In [4]:
# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Evolve binary population

# create a fixed-total-time hardening mechanism
fixed = holo.hardening.Fixed_Time_2PL.from_pop(pop, 2.0 * GYR)
# Create an evolution instance using population and hardening mechanism
evo = holo.evolution.Evolution(pop, fixed)
# evolve binary population
evo.evolve()

AttributeError: module 'holodeck' has no attribute 'population'

Calculate GWB

In [ ]:
#fmin = 1.0 / (20.0*YR)
#fmax = 1.0 / (0.3*YR) * 0.5
#df = fmin
#linfreqs = np.arange(fmin, fmax + df/10.0, df)
#print(f"assigning {linfreqs.size} linfreqs: {linfreqs}")
#print(f"min, max, binsize: {fmin}, {fmax}, {df}")
#print(f"Log frequencies: {np.log10(linfreqs)}")
#print(f"log spacing: {np.diff(np.log10(linfreqs))}")

#print(fmax/fmin)

In [ ]:
# construct sampling frequencies
#freqs = holo.utils.nyquist_freqs(dur=20.0*YR, cad=0.3*YR)

## the code wants evenly log-spaced frequencies, doing this manually for the moment
#fmin = 1.0 / (20.0*YR)
#fmax = 1.0 / (0.3*YR) * 0.5
#freqs = np.logspace(np.log10(fmin), np.log10(fmax), int(fmax/fmin))
#lgdf = np.diff(np.log10(freqs))[0] # log bin width                                                                                             
#lg_freqs_edges = np.log10(freqs) - lgdf/2.0 # shift to edges                                                                                   
#lg_freqs_edges = np.concatenate([lg_freqs_edges, [lg_freqs_edges[-1] + lgdf]])
#freqs_edges = 10.0**lg_freqs_edges


#print(f"Created {freqs.size} evenly log-spaced sampling frequencies in range {fmin} - {fmax}")
#print(f"log min, max: {np.log10(fmin)}, {np.log10(fmax)}")
#print(f"Frequencies: {freqs}")
#print(f"Log frequencies: {np.log10(freqs)}")
#print(f"log spacing: {np.diff(np.log10(freqs))}")
#print(f"lin spacing: {np.diff(freqs)}")
#print(f"lg_freqs_edges: {lg_freqs_edges}")
#print(freqs.size, lg_freqs_edges.size)

# calculate discretized GW signals
#gwb = holo.gravwaves.GW_Discrete(evo, freqs, nreals=1000)
#gwb.emit()

In [ ]:
# construct sampling frequencies
freqs = holo.utils.nyquist_freqs(dur=20.0*YR, cad=0.3*YR, lgspace=True)


In [ ]:
print(freqs)
print(np.log10(freqs))

In [ ]:
# calculate discretized GW signals
gwb = holo.gravwaves.GW_Discrete(evo, freqs, nreals=1000)
gwb.emit()

Plot GWB

In [ ]:
plot.plot_gwb(freqs, gwb.back)
plt.show()

# Population Modifiers (`_Population_Modifier`)

## Resample (`PM_Resample`)

Apply a modifier to resample binary population by some factor, increasing the number of sample points

In [ ]:
mod_resamp = holo.population.PM_Resample(resample=0.5)
old_size = pop.size
pop.modify(mod_resamp)
print(f"Population size increased from {old_size} to {pop.size} elements")

Plot resampled population, now with many more points

In [ ]:
plot_bin_pop(pop)
plt.show()

## Mass Reset (`PM_Mass_Reset`)

Apply another `Modifier` to change the MBH masses to match a particular M-Mbulge relation

In [ ]:
# Create the modifier using M-Mbulge relation
mmbulge = holo.relations.MMBulge_KH2013()
mod_KH2013 = holo.population.PM_Mass_Reset(mmbulge, scatter=True)

# Choose percentiles for reporting statistical properties
percs = 100*sp.stats.norm.cdf([-1, 0, 1])
percs = [0,] + percs.tolist() + [100,]

# Format nicely
str_array = lambda xx: ", ".join(["{:.2e}".format(yy) for yy in xx])
str_masses = lambda xx: str_array(np.percentile(xx/MSOL, percs))

# Modify population
print(f"                {0:7.0f}% {16:7.0f}% {50:7.0f}% {84:7.0f}% {100:7.0f}%")
print("Masses before: ", str_masses(pop.mass))
pop.modify(mod_KH2013)
print("Masses after : ", str_masses(pop.mass))

Plot MBH-Galaxy scaling relationship, showing old and new masses (i.e. before and after modification)

In [ ]:
plot_mbh_scaling_relations(pop)
plt.show()

## Eccentricity (`PM_Eccentricity`)

Demonstrate the eccentricity function

In [ ]:
DEF_VALS = [1.0, 0.2]
NUM = 1e3

edges = np.linspace(0.0, 1.0, 61)
kw = dict(hist=False, density=True, edges=edges)

fig, axes = plt.subplots(figsize=[10, 6], nrows=2)
for ax in axes:
    ax.grid(alpha=0.15)
    
ax = axes[0]
handles = []
labels = []
for cent in [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]:
    vals = [vv for vv in DEF_VALS]
    vals[0] = cent
    ecc = holo.population.PM_Eccentricity(vals)
    xx = ecc._random_eccentricities(NUM)
    hh = kale.dist1d(xx, ax=ax, **kw)
    handles.append(hh)
    labels.append(vals)
    
ax.legend(handles, labels, fontsize=8)


ax = axes[1]
kw = dict(hist=True, density=False, edges=edges)
handles = []
labels = []
for wid in [0.01, 0.05, 0.1, 0.5, 2]:
    vals = [vv for vv in DEF_VALS]
    vals[1] = wid
    ecc = holo.population.PM_Eccentricity(vals)
    xx = ecc._random_eccentricities(NUM)
    hh = kale.dist1d(xx, ax=ax, **kw)
    handles.append(hh)
    labels.append(vals)
    
ax.legend(handles, labels, fontsize=8)

plt.show()

Add eccentricity to binary population, plot all initial binary parameters

In [ ]:
ecc = holo.population.PM_Eccentricity()
pop.modify(ecc)
plot_bin_pop(pop)
plt.show()

# Binary Evolution Models

## Phenomenological 'Fixed Time' Evolution Model

The `holodeck.evolution.Fixed_Time` class uses a simple parametrized hardening model that forces all binaries to coalesce in a given total amount of time.  It uses a double power-law in hardening timescale [i.e. $\tau_a \equiv a / (da/dt) = dt/d\ln a$], with two power-law indices, and a transition binary-separation, as the parameters.  Based on those three parameters, it constructs a complete binary evolution history for each binary (including GW emission), and chooses a normalization such that the total evolution time matches the desired value.

Construct an `Evolution` instance using a `Fixed_Time` model for the given lifetime.

In [ ]:
# Set timescale for all binaries to merge over
fix_time = 2.0 * GYR
# ---- Construct hardening model with this desired merging time
# When this instance is created, it calculates the appropriate hardening normalization for each
# binary to reach the desired coalescing time
fixed = holo.hardening.Fixed_Time_2PL.from_pop(pop, fix_time)
# Construct evolution instance using fixed time hardening
evo = holo.evolution.Evolution(pop, fixed)
# Evolve population
evo.evolve()

Compare resulting lifetimes to the targeted lifetime.  The match won't be perfect, but it should be within a few percent of the desired timescale (`fix_time`).

In [ ]:
# Calculate the total lifetime of each binary
time = evo.tlook
dt = time[:, 0] - time[:, -1]

# Create figure
fig, ax = plot.figax(scale='lin', xlabel='Time: actual/specified', ylabel='density')
# use kalepy to plot distribution
kale.dist1d(dt/fix_time, density=True)

plt.show()

Plot the hardening timescale ($\tau_a$) vs. binary separation ($a$).

In [ ]:
# Create spacing in separation (xaxis) to plot against
sepa = np.logspace(-4, 4, 100) * PC
# Plot hardening rates
plot_evo(evo, sepa=sepa)

plt.show()

Plot the hardening timescale ($\tau_f$) vs. binary frequency ($f$).

In [ ]:
# Create frequency spacing (xaxis) to plot against
freqs = np.logspace(-3, 1, 30) / YR
# plot evolution
plot_evo(evo, freqs=freqs)
plt.show()

Calculate GWB from the `evolution` instance, which takes into account the fixed-time hardening model

In [ ]:
gwb = holo.gravwaves.GW_Discrete(evo, freqs, nreals=5)
gwb.emit()

Plot GWB.

In [ ]:
plot_gwb(gwb)
plt.show()